In [1]:
#Jia Yi (Susan) Wang
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import re
#import seaborn
#seaborn.set()

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

""""from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC"""

"""from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier"""

#from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

In [2]:
genres = pd.read_json('data/genres.json.gz', orient='record', lines=True, encoding='utf-8')
omdb_data = pd.read_json('data/omdb-data.json.gz', orient='record', lines=True, encoding='utf-8')
rt_data = pd.read_json('data/rotten-tomatoes.json.gz', orient='record', lines=True, encoding='utf-8')
wd_data = pd.read_json('data/wikidata-movies.json.gz', orient='record', lines=True, encoding='utf-8')

In [3]:
omdb_data=omdb_data.explode('omdb_genres')
omdb_data=omdb_data[omdb_data.omdb_awards.notnull()]
omdb_data=omdb_data[omdb_data.omdb_awards!='N\A']

nominations_re = re.compile(r'Nominated for (\d+)')
nominations_re2 = re.compile(r'(\d+) nomination(s?)')
wins_re = re.compile(r'(\d+) win(s?)')
wins_re2 = re.compile(r'Won (\d+)')

def get_wins(txt):
    wins = 0
    wins1 = wins_re.search(txt)
    wins2 = wins_re2.search(txt)
    if wins1:
        wins = wins + int(wins1.group(1))
    if wins2:
        wins = wins + int(wins2.group(1))
    return wins

def get_nominations(txt):
    noms = 0
    nominations1 = nominations_re.search(txt)
    nominations2 = nominations_re2.search(txt)
    if nominations1:
        noms = noms + int(nominations1.group(1))
    if nominations2:
        noms = noms + int(nominations2.group(1))
    return noms

omdb_data['nominations'] = omdb_data['omdb_awards'].apply(get_nominations)
omdb_data['wins'] = omdb_data['omdb_awards'].apply(get_wins)
    

In [4]:
#np.set_printoptions(threshold=np.inf)
#set(omdb_data.omdb_awards)
omdb_data

,imdb_id,omdb_genres,omdb_plot,omdb_awards,nominations,wins
0,tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0
0,tt0060814,History,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0
0,tt0060814,War,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0
1,tt2379713,Action,A cryptic message from the past sends James Bo...,Won 1 Oscar. Another 7 wins & 32 nominations.,32,8
1,tt2379713,Adventure,A cryptic message from the past sends James Bo...,Won 1 Oscar. Another 7 wins & 32 nominations.,32,8
1,tt2379713,Thriller,A cryptic message from the past sends James Bo...,Won 1 Oscar. Another 7 wins & 32 nominations.,32,8
2,tt0230575,Comedy,"The makers of this parody of ""Night of the Liv...",N/A,0,0
2,tt0230575,Horror,"The makers of this parody of ""Night of the Liv...",N/A,0,0
3,tt0065988,Adventure,Jack Crabb is 121 years old as the film begins...,Nominated for 1 Oscar. Another 5 wins & 8 nomi...,9,5
3,tt0065988,Comedy,Jack Crabb is 121 years old as the film begins...,Nominated for 1 Oscar. Another 5 wins & 8 nomi...,9,5


In [5]:
wd_data = wd_data.explode('cast_member')
movies_played=wd_data[['cast_member','wikidata_id']].groupby('cast_member').count().reset_index().rename(columns={'wikidata_id':'movies_in'})
wd_data_stars = movies_played[movies_played.movies_in>1]
wd_data_stars = wd_data_stars.set_index('cast_member').join(wd_data.set_index('cast_member'),on='cast_member').reset_index()

In [6]:
wd_data_stars #filter out actors who only performed in one movie

,cast_member,movies_in,wikidata_id,label,imdb_id,rotten_tomatoes_id,enwiki_title,genre,director,publication_date,country_of_origin,original_language,filming_location,based_on,made_profit,metacritic_id,main_subject,series
0,Q100028,2,Q1219862,Die Weiße Rose,tt0084897,m/white_rose,Die Weiße Rose (film),"[Q17013749, Q130232]",[Q67641],1982-01-01,Q183,Q188,NaN,NaN,NaN,NaN,"[Q362, Q8454]",NaN
1,Q100028,2,Q311626,Sophie Scholl – The Final Days,tt0426578,m/sophie_scholl_die_letzten_tage,Sophie Scholl – The Final Days,"[Q645928, Q130232]",[Q67266],2005-02-13,Q183,Q188,NaN,NaN,NaN,movie/sophie-scholl-the-final-days,"[Q362, Q8454]",NaN
2,Q1001175,3,Q325286,David,tt0079028,m/770798780,David (1979 film),[Q130232],[Q73683],1979-01-01,Q183,Q188,NaN,NaN,NaN,NaN,NaN,NaN
3,Q1001175,3,Q1198160,The Magic Mountain,tt0084946,m/thomas_manns_the_magic_mountain,The Magic Mountain (1982 film),"[Q2975633, Q130232]",[Q74013],1982-02-25,Q142,Q188,[Q11911],Q212898,NaN,NaN,NaN,NaN
4,Q1001175,3,Q681602,Love in Thoughts,tt0325733,m/18536,Love in Thoughts,"[Q130232, Q20442589]",[Q85305],2004-01-01,Q183,Q188,NaN,NaN,NaN,NaN,NaN,NaN
5,Q1001214,11,Q1211343,The Beverly Hillbillies,tt0106400,m/1046435-beverly_hillbillies,The Beverly Hillbillies (film),[Q157443],[Q441578],1993-01-01,Q30,Q1860,[Q3476090],NaN,NaN,NaN,NaN,NaN
6,Q1001214,11,Q1619460,Broadway Melody of 1936,tt0026144,m/broadway_melody_of_1936,Broadway Melody of 1936,[Q842256],"[Q918762, Q547078]",1935-01-01,Q30,Q1860,NaN,NaN,NaN,NaN,NaN,NaN
7,Q1001214,11,Q193066,Breakfast at Tiffany's,tt0054698,m/breakfast_at_tiffanys,Breakfast at Tiffany's (film),"[Q860626, Q157443, Q52162262]",[Q56093],1961-01-01,Q30,Q1860,[Q60],Q849930,1.0,movie/breakfast-at-tiffanys,NaN,NaN
8,Q1001214,11,Q229547,Born to Dance,tt0027387,m/born_to_dance,Born to Dance,[Q842256],[Q918762],1936-01-01,Q30,Q1860,NaN,NaN,NaN,NaN,NaN,NaN
9,Q1001214,11,Q1093219,"Davy Crockett, King of the Wild Frontier",tt0047977,m/davy_crockett_king_of_the_wild_frontier,"Davy Crockett, King of the Wild Frontier",[Q2143665],[Q2409726],1955-05-25,Q30,Q1860,"[Q99, Q1454]",NaN,NaN,NaN,[Q638530],Q3019192


In [7]:
joined = omdb_data.set_index('imdb_id').join(rt_data.set_index('imdb_id'),on='imdb_id')
joined = joined.join(wd_data_stars.set_index('imdb_id'), lsuffix='_joined', rsuffix='_wd', on='imdb_id')

In [8]:
joined

,omdb_genres,omdb_plot,omdb_awards,nominations,wins,audience_average,audience_percent,audience_ratings,critic_average,critic_percent,...,director,publication_date,country_of_origin,original_language,filming_location,based_on,made_profit,metacritic_id,main_subject,series
imdb_id,,,,,,,,,,,,,,,,,,,,,
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN
tt0060814,Drama,"In this sprawling, star-laden film, we see the...",Nominated for 2 Oscars. Another 2 nominations.,4,0,3.5,72.0,1311.0,6.4,80.0,...,[Q313190],1966-01-01,Q142,Q150,"[Q2875572, Q141747, Q2741353, Q1327827, Q34510...",NaN,NaN,NaN,[Q362],NaN


In [9]:
info = joined.reset_index()
#info = info[['audience_average', 'omdb_genres', 'cast_member', 'wins', 'nominations']]
#info = info[['audience_average', 'omdb_genres', 'cast_member']]
info = info[['audience_average', 'omdb_genres', 'wins', 'nominations']] #tried using cast_member, but ran into overfitting problems
info = info[info.audience_average.notnull()]
#info = info[info.cast_member.notnull()]
info = info[info.omdb_genres.notnull()]
info = info[info.omdb_genres != 'N/A']

#info = info[['audience_average', 'omdb_genres', 'audience_percent', 'audience_ratings', 'critic_average', 'critic_percent']]
#info = info[['audience_average', 'omdb_genres']]
#info=info[info.audience_average.notnull() & info.omdb_genres.notnull()]
#info=info[info.audience_percent.notnull() & info.audience_ratings.notnull()]
#info=info[info.critic_average.notnull() & info.critic_percent.notnull()]

#info = info[['audience_average', 'cast_member']]
#info=info[info.audience_average.notnull() & info.cast_member.notnull()]

In [10]:
info

,audience_average,omdb_genres,wins,nominations
0,3.5,Drama,0,4
1,3.5,Drama,0,4
2,3.5,Drama,0,4
3,3.5,Drama,0,4
4,3.5,Drama,0,4
5,3.5,Drama,0,4
6,3.5,Drama,0,4
7,3.5,Drama,0,4
8,3.5,Drama,0,4
9,3.5,Drama,0,4


In [11]:
#One hot encoder example: https://medium.com/@contactsunny/label-encoder-vs-one-hot-encoder-in-machine-learning-3fc273365621
#Column Transforer source: https://datascience.stackexchange.com/questions/41113/deprecationwarning-the-categorical-features-keyword-is-deprecated-in-version

#Encoding the genres to binary

ct = ColumnTransformer(
    transformers=[('one_hot_encoder', OneHotEncoder(), [0])], #('one_hot_encoder2', OneHotEncoder(handle_unknown='ignore'), [1])],
    remainder='passthrough' 
)

#X = ct.fit_transform(X)

In [12]:
model = Pipeline(
        steps=[('col_trans',ct), ('lr', LinearRegression(fit_intercept=False))]
    )

X=info.drop(columns=['audience_average'],axis=1)
y=info['audience_average']

X_train, X_valid, y_train, y_valid = train_test_split(X,y)

model.fit(X_train, y_train)

print(model.score(X_train,y_train))
print(model.score(X_valid,y_valid))

0.17585985418801175
0.17240670824947724


In [13]:
""""#Looking only at actors
joined2 = joined.join(wd_data_stars.set_index('imdb_id'), lsuffix='_joined', rsuffix='_wd', on='imdb_id') #only include actors who starred in more than one movie
info = joined2.reset_index()
info = info[['audience_average', 'cast_member']]
info = info[info.audience_average.notnull()]
info = info[info.cast_member.notnull()]

X=info.drop(columns=['audience_average'],axis=1)
y=info['audience_average'].values

ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(), [0])],
    remainder='passthrough' 
)

X = ct.fit_transform(X)

X_train, X_valid, y_train, y_valid = train_test_split(X,y)

model = make_pipeline(
        LinearRegression(fit_intercept=False)
    )

model.fit(X_train, y_train)

print(model.score(X_train,y_train))
print(model.score(X_valid,y_valid))"""

'"#Looking only at actors\njoined2 = joined.join(wd_data_stars.set_index(\'imdb_id\'), lsuffix=\'_joined\', rsuffix=\'_wd\', on=\'imdb_id\') #only include actors who starred in more than one movie\ninfo = joined2.reset_index()\ninfo = info[[\'audience_average\', \'cast_member\']]\ninfo = info[info.audience_average.notnull()]\ninfo = info[info.cast_member.notnull()]\n\nX=info.drop(columns=[\'audience_average\'],axis=1)\ny=info[\'audience_average\'].values\n\nct = ColumnTransformer(\n    [(\'one_hot_encoder\', OneHotEncoder(), [0])],\n    remainder=\'passthrough\' \n)\n\nX = ct.fit_transform(X)\n\nX_train, X_valid, y_train, y_valid = train_test_split(X,y)\n\nmodel = make_pipeline(\n        LinearRegression(fit_intercept=False)\n    )\n\nmodel.fit(X_train, y_train)\n\nprint(model.score(X_train,y_train))\nprint(model.score(X_valid,y_valid))'

In [14]:
""""#Looking only at award wins and nominations
omdb_data2=omdb_data[omdb_data.omdb_awards.notnull()]
omdb_data2=omdb_data2[omdb_data.omdb_awards!='N\A']

nominations_re = re.compile(r'Nominated for (\d+)')
nominations_re2 = re.compile(r'(\d+) nomination(s?)')
wins_re = re.compile(r'(\d+) win(s?)')
wins_re2 = re.compile(r'Won (\d+)')

def get_wins(txt):
    wins = 0
    wins1 = wins_re.search(txt)
    wins2 = wins_re2.search(txt)
    if wins1:
        wins = wins + int(wins1.group(1))
    if wins2:
        wins = wins + int(wins2.group(1))
    return wins

def get_nominations(txt):
    noms = 0
    nominations1 = nominations_re.search(txt)
    nominations2 = nominations_re2.search(txt)
    if nominations1:
        noms = noms + int(nominations1.group(1))
    if nominations2:
        noms = noms + int(nominations2.group(1))
    return noms

omdb_data2['nominations'] = omdb_data2['omdb_awards'].apply(get_nominations)
omdb_data2['wins'] = omdb_data2['omdb_awards'].apply(get_wins)

joined3 = omdb_data2.set_index('imdb_id').join(rt_data.set_index('imdb_id'),on='imdb_id')

info = joined3.reset_index()
info = info[['audience_average', 'wins', 'nominations']]
info = info[info.audience_average.notnull()]

X=info.drop(columns=['audience_average'],axis=1)
y=info['audience_average'].values

X_train, X_valid, y_train, y_valid = train_test_split(X,y)

model = make_pipeline(
        LinearRegression(fit_intercept=False)
    )

model.fit(X_train, y_train)

print(model.score(X_train,y_train))
print(model.score(X_valid,y_valid)) #Looking only at genres"""

'"#Looking only at award wins and nominations\nomdb_data2=omdb_data[omdb_data.omdb_awards.notnull()]\nomdb_data2=omdb_data2[omdb_data.omdb_awards!=\'N\\A\']\n\nnominations_re = re.compile(r\'Nominated for (\\d+)\')\nnominations_re2 = re.compile(r\'(\\d+) nomination(s?)\')\nwins_re = re.compile(r\'(\\d+) win(s?)\')\nwins_re2 = re.compile(r\'Won (\\d+)\')\n\ndef get_wins(txt):\n    wins = 0\n    wins1 = wins_re.search(txt)\n    wins2 = wins_re2.search(txt)\n    if wins1:\n        wins = wins + int(wins1.group(1))\n    if wins2:\n        wins = wins + int(wins2.group(1))\n    return wins\n\ndef get_nominations(txt):\n    noms = 0\n    nominations1 = nominations_re.search(txt)\n    nominations2 = nominations_re2.search(txt)\n    if nominations1:\n        noms = noms + int(nominations1.group(1))\n    if nominations2:\n        noms = noms + int(nominations2.group(1))\n    return noms\n\nomdb_data2[\'nominations\'] = omdb_data2[\'omdb_awards\'].apply(get_nominations)\nomdb_data2[\'wins\'] = o